In [1]:
from keras.models import Sequential, Model, load_model
from keras.utils.np_utils import to_categorical
from keras.layers import Flatten, Dense, Dropout, Reshape, Permute, Activation, BatchNormalization, \
    Input
from keras.engine.topology import merge
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback
import numpy as np
from keras.wrappers.scikit_learn import KerasRegressor
from keras.applications.resnet50 import ResNet50
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# from image_util import crop_center
import pickle as pkl
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
# load data
Xtrain = np.load('./data/dataset256/Xtrain.npy')
Xtest = np.load('./data/dataset256/Xtest.npy')
Ytrain_reg = np.load('./data/dataset256/Ytrain_reg.npy')
Ytest_reg = np.load('./data/dataset256/Ytest_reg.npy')
Ytrain_cls = np.load('./data/dataset256/Ytrain_cls.npy')
Ytest_cls = np.load('./data/dataset256/Ytest_cls.npy')

In [3]:
print ('Xtrain', Xtrain.shape)
print ('Xtest', Xtest.shape)
print ('Ytrain_reg', Ytrain_reg.shape)
print ('Ytest_reg', Ytest_reg.shape)
print ('Ytrain_cls', Ytrain_cls.shape)
print ('Ytest_cls', Ytest_cls.shape)

Xtrain (15405, 256, 256, 3)
Xtest (3852, 256, 256, 3)
Ytrain_reg (15405,)
Ytest_reg (3852,)
Ytrain_cls (15405,)
Ytest_cls (3852,)


In [4]:
# netowrk settings

batch_size = 64
nb_epoch = 2
folder = './myresnet-cls-training-checkpoint/'
load_path = False
n_cls = int(np.max(Ytrain_cls))


In [5]:
Ytrain_cls = to_categorical(Ytrain_cls-1)
Ytest_cls = to_categorical(Ytest_cls-1)

In [6]:
def addResidualBlock(inp, size=(3,3), subsample=(4,4), filters_conv_rate=2, name=None):
    conv_filters = int(inp.get_shape()[-1])*filters_conv_rate
    inp = BatchNormalization()(inp)
    first = Convolution2D(conv_filters, size[0], size[1], border_mode='same', subsample=subsample, activation='relu')(inp)
    
    second = BatchNormalization()(first)
    second = Convolution2D(conv_filters, 3,3, border_mode='same', activation='relu')(second)
    
    second = BatchNormalization()(second)
    second = Convolution2D(conv_filters, 3,3, border_mode='same', activation='relu')(second)
    
    return merge([first, second], mode='sum', name=name)

In [7]:
def MyResNetRegr():
    inp = Input(shape=(256, 256,3))
    out = Convolution2D(16, 3,3, border_mode='same', activation='relu', subsample=(4,4))(inp)
    for i in range(3):
        out = addResidualBlock(out)
        out = BatchNormalization()(out)
        out = Activation('relu')(out)
    
    out = BatchNormalization()(out)
    out = Flatten()(out)
    out = Dense(128, activation='relu', name='fcc')(out)
    out = Dense(1, name='out')(out)
    
    model = Model(input=inp, output=out)
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def MyResNetCls():
    inp = Input(shape=(256, 256,3))
    out = Convolution2D(16, 3,3, border_mode='same', activation='relu', subsample=(4,4))(inp)
    for i in range(3):
        out = addResidualBlock(out)
        out = BatchNormalization()(out)
        out = Activation('relu')(out)
    
    out = BatchNormalization()(out)
    out = Flatten()(out)
    out = Dense(128, activation='relu', name='fcc')(out)
    out = Dense(n_cls, activation='softmax', name='out')(out)
    
    model = Model(input=inp, output=out)
    
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model


def MyResNet2Obj():
    inp = Input(shape=(256, 256,3))
    out = Convolution2D(16, 3,3, border_mode='same', activation='relu', subsample=(4,4))(inp)
    for i in range(3):
        out = addResidualBlock(out)
        out = BatchNormalization()(out)
        out = Activation('relu')(out)
    
    out = BatchNormalization()(out)
    out = Flatten()(out)
    out = Dense(128, activation='relu', name='fcc')(out)
    out_reg = Dense(1, name='out_reg')(out)
    out_cls = Dense(n_cls, activation='softmax', name='out_cls')(out)
    
    model = Model(input=inp, output=[out_reg, out_cls])
    model_reg = Model(input=inp, output=out_reg)
    model_cls = Model(input=inp, output=out_cls)

    model.compile(loss=['mean_squared_error', 'categorical_crossentropy'], optimizer='adam', metrics=['accuracy'])
    model_reg.compile(loss='mean_squared_error', optimizer='adam')
    model_cls.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model, model_reg, model_cls


In [ ]:
if load_path:
    model = load_model(load_path)
else:
    model = MyResNetRegr()
model.summary()

In [ ]:
if load_path:
    model = load_model(load_path)
else:
    model = MyResNetCls()
model.summary()

In [8]:
if load_path:
    model = load_model(load_path)
else:
    model, model_reg, model_cls = MyResNet2Obj()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 256, 256, 3)   0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 64, 16)    448         input_1[0][0]                    
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 64, 64, 16)    64          convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 16, 16, 32)    4640        batchnormalization_1[0][0]       
___________________________________________________________________________________________

In [9]:
# batch logger 
# ref: https://github.com/fchollet/keras/issues/2850#issuecomment-222542429
class NBatchLogger(Callback):
    def __init__(self,display=100):
        '''
        display: Number of batches to wait before outputting loss
        '''
        self.seen = 0
        self.display = display

    def on_batch_end(self,batch,logs={}):
        self.seen += logs.get('size', 0)
        if self.seen % self.display == 0:
            print('\n{0}/{1} - Batch Loss: {2}'.format(self.seen,self.params['nb_sample'],
                                                self.params['metrics'][0]))

In [10]:
#train just 1000 samples, 3 epochs
# if False:
checkpoint = ModelCheckpoint(filepath=folder + 'checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5')
out_batch = NBatchLogger(display=1)
history = model.fit(x=Xtrain[:1000], y=[Ytrain_reg[:1000], Ytrain_cls[:1000]], 
          nb_epoch=nb_epoch, batch_size=batch_size,validation_split=0.1, verbose=1, callbacks=[checkpoint]) 

Train on 900 samples, validate on 100 samples
Epoch 1/2
900/900 [==============================] - 15s - loss: 2.4099 - out_reg_loss: 0.4321 - out_cls_loss: 1.9778 - out_reg_acc: 0.0000e+00 - out_cls_acc: 0.2156 - val_loss: 11.7482 - val_out_reg_loss: 3.7271 - val_out_cls_loss: 8.0210 - val_out_reg_acc: 0.0000e+00 - val_out_cls_acc: 0.1000
Epoch 2/2
900/900 [==============================] - 9s - loss: 1.8553 - out_reg_loss: 0.2901 - out_cls_loss: 1.5652 - out_reg_acc: 0.0000e+00 - out_cls_acc: 0.4044 - val_loss: 12.3423 - val_out_reg_loss: 1.3051 - val_out_cls_loss: 11.0372 - val_out_reg_acc: 0.0000e+00 - val_out_cls_acc: 0.1000


In [11]:
model_reg.save(folder + 'model_reg.hdf5')
model_cls.save(folder + 'model_cls.hdf5')

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
evl = model.evaluate(x=Xtest, y=Ytest,batch_size=batch_size)

In [ ]:
evl

In [ ]:
# validation keras fix 
# ref: http://stackoverflow.com/questions/41796618/python-keras-cross-val-score-error/41841066#41841066
from keras.wrappers.scikit_learn import BaseWrapper
import copy

def custom_get_params(self, **params):
    res = copy.deepcopy(self.sk_params)
    res.update({'build_fn': self.build_fn})
    return res

BaseWrapper.get_params = custom_get_params

In [ ]:
# regression 
# rev: http://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/

In [ ]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=MyNet, nb_epoch=nb_epoch, batch_size=batch_size, verbose=1)

In [ ]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, Xtrain[:1000], Ytrain[:1000], cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))